In [ ]:
import speech_recognition as sr
import pyttsx3
import pyaudio
import torch
from transformers import BlenderbotForConditionalGeneration, BlenderbotTokenizer

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def list_microphones():
    print("Available Microphones:", sr.Microphone.list_microphone_names())
    p = pyaudio.PyAudio()
    for i in range(p.get_device_count()):
        print(p.get_device_info_by_index(i))

def recognize_speech():
    recognizer = sr.Recognizer()
    mic_index = 0  # Change based on available microphones
    mic = sr.Microphone(device_index=mic_index)
    
    with mic as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print("Listening...")
        try:
            audio = recognizer.listen(source, timeout=10, phrase_time_limit=15)
            text = recognizer.recognize_google(audio)
            print("You said:", text)
            return text
        except sr.UnknownValueError:
            print("Could not understand audio, please try again.")
            return None
        except sr.RequestError:
            print("Error with speech recognition service. Check your internet connection.")
            return None
        except Exception as e:
            print("Unexpected error:", str(e))
            return None

# Load Blenderbot model for chatbot
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def generate_response(text):
    if not text:
        return "I didn't catch that. Could you repeat?"
    inputs = tokenizer(text, return_tensors="pt").to(device)
    output = model.generate(**inputs)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def chatbot():
    print("AI is now actively listening. Speak to start a conversation.")
    while True:
        user_input = recognize_speech()
        if not user_input:
            continue
        
        response = generate_response(user_input)
        print("AI:", response)
        speak(response)

if __name__ == "__main__":
    chatbot()
